# 02 RAG | 01 Chat Completion | 04 Create Image Completion

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The Azure OpenAI client from the Azure.AI.OpenAI NuGet package offers a unified interface to interact with various specialized clients, each designed to handle specific tasks. 

These include clients for chat completion, embeddings, and the agent framework, among others, enabling developers to easily integrate and utilize OpenAI's capabilities for generating text, understanding context, and building intelligent applications within the Azure ecosystem.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Images;
using DotNetEnv;
using System.IO;
using System.Text.Json; 
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string imageDeploymentName = Environment.GetEnvironmentVariable("WS_IMAGE_DEPLOYMENTNAME") ?? "WS_IMAGE_DEPLOYMENTNAME not found";
string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);
ImageClient imageClient = azureOpenAIClient.GetImageClient(imageDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"ImageClient created...");

**Expected output:**

Installed Packages

  - Azure.AI.OpenAI, 2.0.0-beta.2
  - DotNetEnv, 2.5.0

AzureOpenAI Client created...

ImageClient created...

## Step 2: Create Image

The following cell, demonstrate a basic interaction using the `imageClient` instance. In the sample a photo realistic image is created. 

System prompt: `"Create a photo realistic image of an US american football team winning the Super Bowl. 
    Show them celebrating as team in a stadium with confetti falling from the sky"`

The `imageGenerationOptions' allow more fine granular control how the image is created and in which format it is returned to the caller.

In [ ]:
using System.Net.Http;

//Generate Image
string imagePrompt = @"
    Create a photo realistic image of an US american football team winning the Super Bowl. 
    Show them celebrating as team in a stadium with confetti falling from the sky
";
string imageFileName = Path.Join(assetsFolder, "Image", "SuperBowl_Photorealistic.png");

ImageGenerationOptions imageGenerationOptions = new ImageGenerationOptions() {
    Quality = GeneratedImageQuality.Standard,
    ResponseFormat = GeneratedImageFormat.Bytes, 
    Size = GeneratedImageSize.W1024xH1024,
    Style = GeneratedImageStyle.Natural
};

ClientResult<GeneratedImage> result = await imageClient.GenerateImageAsync(
    imagePrompt, 
    imageGenerationOptions
);

Console.WriteLine($"Image generated and saved to {imageFileName}...");         


**Expected result:**

Image generated and saved to ../../assets/Image/SuperBowl_Photorealistic.png...